#### Collaborative Filtering Technique
* Nearest Neighbours Model - use ratings of most similar user

In [1]:
import pandas as pd

In [2]:
dataFile = "BX-CSV-Dump/BX-Book-Ratings.csv"
data = pd.read_csv(dataFile, sep=";", header=0, names=["user", "isbn", "rating"], encoding = "ISO-8859-1")

In [3]:
#Rating in the range of 0-10
data.head(5)

,user,isbn,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


### Book Meta data

In [4]:
bookFile = "BX-CSV-Dump/BX-Books.csv"
books = pd.read_csv(bookFile, sep=";", header=0, error_bad_lines=False, usecols=[0, 1, 2], index_col=0, names=["isbn", "title", "author"], encoding = "ISO-8859-1")

In [5]:
books.head()

,title,author
isbn,,
0195153448,Classical Mythology,Mark P. O. Morford
0002005018,Clara Callan,Richard Bruce Wright
0060973129,Decision in Normandy,Carlo D'Este
0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
0393045218,The Mummies of Urumchi,E. J. W. Barber


In [6]:
def bookMeta(isbn):
    title = books.at[isbn, "title"]
    author = books.at[isbn, "author"]
    return title, author

In [7]:
bookMeta("0195153448")

('Classical Mythology', 'Mark P. O. Morford')

#### Top N books for user

In [ ]:
def faveBooks(user, N):
    userRatings = data[data["user"]==user] #Filter data relevant to user
    sortedRatings = pd.DataFrame.sort_values(userRatings, by=['rating'], ascending=[0])[:N]
    sortedRatings["title"] = sortedRatings["isbn"].apply(bookMeta)
    return sortedRatings

In [ ]:
#Considering the books which are there in the books df
data = data[data["isbn"].isin(books.index)]

In [ ]:
faveBooks(204622, 5)

# Construct rating matrix

In [ ]:
# 1 million ratings
data.shape

Before converting data to desired matrix usersPerISBN, ISBNsPerUser for remove some unusual occuring data
- Take subset the data to only those ISBNs which have been read by more than 10 users
- Take subset the data to only users which have read more than 10 books

In [ ]:
usersPerISBN = data.isbn.value_counts()

In [ ]:
#col1: isbns col2: user count
usersPerISBN.head(10)

In [ ]:
#These many unique isbn
#Which means that the rating matrix will end up having these many columns
usersPerISBN.shape

In [ ]:
ISBNsPerUser = data.user.value_counts()

In [ ]:
#col1: userId col2: isbns count
ISBNsPerUser.head(10)

In [ ]:
#Number of users
#Which means that the rating matrix will end up having these many rows
ISBNsPerUser.shape

In [ ]:
# 105283 x 340556
# Resultant matrix shape

In [ ]:
# Take subset the data to only those ISBNs which have been read by more than 10 users
data = data[data["isbn"].isin(usersPerISBN[usersPerISBN>10].index)]
data.shape

In [ ]:
# Take subset the data to only users which have read more than 10 books
data = data[data["user"].isin(ISBNsPerUser[ISBNsPerUser>10].index)]
data.shape

In [ ]:
data.columns

In [ ]:
userItemRatingMatrix = pd.pivot_table(data, values='rating', index=['user'], columns=['isbn'])

In [ ]:
userItemRatingMatrix.head()

In [ ]:
#Removes the density of matrix Memory efficient
from scipy.sparse import coo_matrix
data['user'] = data['user'].astype("category")
data['isbn'] = data['isbn'].astype("category")

In [ ]:
R = coo_matrix(
            (data['rating'].astype(float),
                  (data['user'].cat.codes.copy(),
                   data['isbn'].cat.codes.copy()
                  )
            )
          )

In [ ]:
R.shape

In [ ]:
len(R.data)

In [ ]:
R.row[0], R.row[1], R.row[2], R.row[88434]

In [ ]:
R.col[0], R.col[2], R.col[1]

In [ ]:
R.data[90]

In [ ]:
len(R.row)

### Intialize factor matrices

In [ ]:
M, N = R.shape
K = 3 #Factors

In [ ]:
import numpy as np
P = np.random.rand(M, K)
Q = np.random.rand(K, M)

### Compute the error

In [ ]:
from numpy.linalg import norm

def error(R, P, Q, lamda=0.02):
    ratings = R.data
    rows = R.row
    cols = R.col
    e = 0
    for ui in range(len(ratings)):
        rui = ratings[ui]
        u = rows[ui]
        i = cols[ui]
        if rui > 0:
            e = e + pow(rui-np.dot(P[u, :], Q[:, i]), 2) + \
                lamda*(pow(norm(P[u, :]), 2)) + 